In [ ]:
import numpy as np
from phonopy import Phonopy
from phonopy.structure.atoms import PhonopyAtoms
from phonopy.phonon.band_structure import get_band_qpoints_and_path_connections
from scipy.interpolate import interp1d
import ase, ase.build, ase.io
from ase.visualize import view
from ase.calculators.castep import Castep
from glob import glob
import os
from quippy.potential import Potential
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] =12,10
plt.rcParams.update({'font.size': 22})
%matplotlib inline

top = '/home/physics/phspnt/project_work/phonnopy_tut'

class AtomsToPhonons:
    def __init__(self, primitive_cell, phonon_grid, displacement, kpath, calculator, kpoints=100, castep=False, plusminus=False,diagonal=True):
        self.calculator_string=calculator
        self.get_supercell(primitive_cell, phonon_grid, displacement, plusminus,diagonal)
        if castep==False: self.get_forces_gap()
        if castep==True: self.get_forces_castep()
        self.get_band_struct(kpath, kpoints)   
    def get_supercell(self, primitive_cell, phonon_grid, displacement, plusminus,diagonal):
        self.unitcell = PhonopyAtoms(symbols=primitive_cell.get_chemical_symbols(),
                        cell=primitive_cell.get_cell(),
                        scaled_positions=primitive_cell.get_scaled_positions()
                       )
        self.phonon = Phonopy(self.unitcell,phonon_grid)
        self.phonon.generate_displacements(distance=displacement, is_plusminus=plusminus, is_diagonal=diagonal)
        self.supercells = self.phonon.supercells_with_displacements        
    def get_forces_gap(self):
        potential= Potential(param_filename=self.calculator_string[0])
        forces = []
        self.atoms=[]
        for i, s in enumerate(self.supercells) :
            a=ase.Atoms(symbols=s.get_chemical_symbols(), cell=s.get_cell(), 
                scaled_positions=s.get_scaled_positions(), pbc=True)
            self.atoms.append(a)
            a.set_calculator(potential)
            forces.append(a.get_forces())
        self.forces = forces
    def get_forces_castep(self):
        forces=[]
        self.atoms=[]
        for i, s in enumerate(self.calculator_string):
            castep_atoms = ase.io.read(s)
            self.atoms.append(castep_atoms)
            forces.append(castep_atoms.get_forces())
        self.forces = forces    
    def get_band_struct(self, kpath, kpoints):
        self.phonon.set_forces(self.forces)
        self.phonon.produce_force_constants()
        qpoints, connections = get_band_qpoints_and_path_connections(kpath, npoints=kpoints)
        self.phonon.run_band_structure(qpoints, path_connections=connections) 
        self.frequencies_array = self.phonon.get_band_structure_dict()['frequencies']
        self.frequencies=self.frequencies_array[0]
        xticks=[]
        x=0
        for i, val in enumerate(self.frequencies_array[1:]):
            self.frequencies = np.append(self.frequencies, val, axis=0)
            x+=len(val)
            xticks.append(x)
        n_kpoints = kpoints*len(kpath[0])
        n_trace = int(n_kpoints / (len(kpath[0])))
        self.normal_ticks = [i*n_trace for i in range(len(kpath[0]))]

class AtomsToPDOS:
    def __init__(self, primitive_cell, phonon_grid, displacement, calculator, kpoints=100, castep=False, plusminus=False,diagonal=True):
        self.calculator_string=calculator
        self.get_supercell(primitive_cell, phonon_grid, displacement, plusminus,diagonal)
        if castep==False: self.get_forces_gap()
        if castep==True: self.get_forces_castep()
        self.get_dynamical_matrix()
    def get_supercell(self, primitive_cell, phonon_grid, displacement, plusminus,diagonal):
        self.unitcell = PhonopyAtoms(symbols=primitive_cell.get_chemical_symbols(),
                        cell=primitive_cell.get_cell(),
                        scaled_positions=primitive_cell.get_scaled_positions()
                       )
        self.phonon = Phonopy(self.unitcell,phonon_grid)
        self.phonon.generate_displacements(distance=displacement, is_plusminus=plusminus, is_diagonal=diagonal)
        self.supercells = self.phonon.supercells_with_displacements        
    def get_forces_gap(self):
        potential= Potential(param_filename=self.calculator_string[0])
        forces = []
        self.atoms=[]
        for i, s in enumerate(self.supercells) :
            a=ase.Atoms(symbols=s.get_chemical_symbols(), cell=s.get_cell(), 
                scaled_positions=s.get_scaled_positions(), pbc=True)
            self.atoms.append(a)
            a.set_calculator(potential)
            forces.append(a.get_forces())
        self.forces = forces
    def get_forces_castep(self):
        forces=[]
        self.atoms=[]
        for i, s in enumerate(self.calculator_string):
            castep_atoms = ase.io.read(s)
            self.atoms.append(castep_atoms)
            forces.append(castep_atoms.get_forces())
        self.forces = forces   
    def get_dynamical_matrix(self):
        self.phonon.set_forces(self.forces)
        self.phonon.produce_force_constants()
    def get_PDOS(self, qmesh):
        self.phonon.run_mesh(qmesh, with_eigenvectors=True, is_mesh_symmetry=False)
        self.phonon.run_projected_dos()
        self.pdos = self.phonon.get_projected_dos_dict()
    def get_DOS(self, qmesh):
        self.phonon.run_mesh(qmesh)
        self.phonon.run_total_dos()
        self.dos = self.phonon.get_total_dos_dict()

class PhononFromCastep:

    def __init__(self, castep_file, kpath, frequency_mask):
        self.number_of_branches = frequency_mask.shape[1]
        self.get_frequencies(castep_file)
        self.in_path = np.array(kpath[0], dtype=float)
        self.get_kpath(castep_file)
        self.find_index()
        self.rescale_xaxis()
    def __str__(self):
        return f"Get Phonon from Castep file object"
    def get_frequencies(self, castep_file):
        grep_mod= '-A'+str(self.number_of_branches+1)
        dummy = !grep $grep_mod '(THz)' $castep_file | sed "/THz/d; /^ + *+$/d; /^--/d" 
        new_list = [i.split()[2] for i in dummy]
        #dummy = !grep $grep_mod '(THz)' $castep_file | sed "/THz/d; /^ + *+$/d; /^--/d" | awk '{print $3}'        
        frequencies =  np.array(new_list, dtype=float)    
        self.kpoints = int((len(frequencies)/self.number_of_branches))
        self.frequencies = np.reshape(frequencies, [self.kpoints, self.number_of_branches])
    
    def get_kpath(self,castep_file):
        header_string= "'Performing frequency calculation at  " + str(self.kpoints) + " wavevectors (q-pts)'"
        header_string="{0}".format(header_string)
        lines_after_header = "-A"+str(self.kpoints +1)
        raw_string =!grep 'q-pt' $castep_file | grep $lines_after_header $header_string | tail -$self.kpoints
        proc_string = []
        for i,val in enumerate(raw_string):
            temp= val.split()[4:7]
            temp[2] = temp[2].replace(")",  "")
            proc_string.append(temp)   
        self.kpath=np.array(proc_string, dtype=float)
    def find_index(self):
        j=0
        tol=1e-5
        sympoint_idx=[]
        self.kpath_idx=[]
        for i, val in enumerate(self.kpath):
            #print(abs(np.linalg.norm(self.in_path[j]-val)))
            if abs(np.linalg.norm(self.in_path[j]-val)) < tol:
                sympoint_idx.append(i)
                j=j+1
        #print(sympoint_idx)
        
        for i in range(len(sympoint_idx)-1):
            x = [j for j in range(sympoint_idx[i], sympoint_idx[i+1]+1)]
            #print(x)
            self.kpath_idx.append(x)
        
    def rescale_xaxis(self): #
        xsplit = 1/len(self.kpath_idx)
        xscale=[0]
        pos=0
        kpath_cut=[]
        kpath_cut.append(self.kpath_idx[0])
        for i in range(len(self.kpath_idx)-1):
            kpath_cut.append(self.kpath_idx[i+1][1:])
        x_inc = xsplit/(len(kpath_cut[0])-1)
        for j in range(len(kpath_cut[0])-1):
            pos = pos + x_inc
            xscale.append(pos)        
        for i in range(len(kpath_cut)-1):
            x_inc = xsplit/(len(kpath_cut[i+1]))
            for j in range(len(kpath_cut[i+1])):
                pos = pos + x_inc
                xscale.append(pos)
        self.xscale = np.array(xscale)

        
class PhononFromCastepPDOS:

    def __init__(self, castep_file : str, atoms_in=None):
        self.number_of_branches = len(atoms_in)*3
        self.get_frequencies(castep_file)
    def __str__(self):
        return f"Get Phonon from Castep file object"
    def get_frequencies(self, castep_file):
        grep_mod= '-A'+str(self.number_of_branches+1)
        dummy = !grep $grep_mod '(THz)' $castep_file | sed "/THz/d; /^ + *+$/d; /^--/d" 
        new_list = [i.split()[2] for i in dummy]        
        frequencies =  np.array(new_list, dtype=float)    
        self.kpoints = int((len(frequencies)/self.number_of_branches))
        self.frequencies = np.reshape(frequencies, [self.kpoints, self.number_of_branches])        

        
def get_error(atp, pfc, as_percentage=False):
    branches = atp.frequencies.shape[1]
    error = [] 
    for i in range(branches):
        xgap = np.linspace(0,max(pfc.xscale),len(atp.frequencies))
        gap_inter = interp1d(xgap, atp.frequencies[:,i], kind='cubic')
        dft_inter = interp1d(pfc.xscale, pfc.frequencies[:,i], kind='cubic')        
        if as_percentage: error.append(100*(dft_inter(xgap) -gap_inter(xgap))/dft_inter(xgap))
        else: error.append(dft_inter(xgap) - gap_inter(xgap))
    return np.array(error).T

def get_cumsum(gap_freq,castep_freq):
    cumsum = np.cumsum(abs(get_error(gap_freq,castep_freq)), axis=0)
    return cumsum

        
#central difference
def get_gamma_v(omega, omega_dv, omega_mdv, v, dv):
    
    
    
    gamma_v = -(v/omega) * (omega_dv - omega_mdv)/(2*dv)
    return gamma_v

def get_gamma_v_bkwd(omega, omega_mdv, v, dv):
    gamma_v = -(v/omega) * (omega - omega_mdv)/(dv)
    return gamma_v            
            
def get_gamma_v_interpolate(pfc, pfc_pdv, pfc_ndv, v, dv):
    rescale = np.linspace(0,1 - 1e-8,100)
    f = interp1d(pfc.xscale, pfc.frequencies[:,:], axis=0)
    f1 = interp1d(pfc_pdv.xscale, pfc_pdv.frequencies[:,:], axis=0)
    f2 = interp1d(pfc_ndv.xscale, pfc_ndv.frequencies[:,:], axis=0)#weird bug...
    omega = f(rescale) ; omega_dv = f1(rescale) ; omega_mdv = f2(rescale)
    gamma_v = -(v/omega) * (omega_dv - omega_mdv)/(2*dv)
    return gamma_v, rescale

class PhononFromCastepPDOS:

    def __init__(self, castep_file : str, atoms_in=None):
        self.number_of_branches = len(atoms_in)*3
        self.get_frequencies(castep_file)
    def __str__(self):
        return f"Get Phonon from Castep file object"
    def get_frequencies(self, castep_file):
        grep_mod= '-A'+str(self.number_of_branches+1)
        dummy = !grep $grep_mod '(THz)' $castep_file | sed "/THz/d; /^ + *+$/d; /^--/d" 
        new_list = [i.split()[2] for i in dummy]
        dummy =!grep 'q-pt' $castep_file
        weights = [i.split()[-2] for i in dummy[1:]]
        self.weights=np.array(weights, dtype=float) 
        frequencies =  np.array(new_list, dtype=float)    
        self.kpoints = int((len(frequencies)/self.number_of_branches))
        self.frequencies = np.reshape(frequencies, [self.kpoints, self.number_of_branches])        

In [ ]:
#Points in BZ, symmetry specific.
GAM=[0, 0, 0]
X=[0.5, 0, 0]
L=[1/4, 1/4, 1/4]
K=[3/8,3/8,0]
W=[1/2,1/4,0]
U=[1/2,1/8,1/8]

#walk in BZ
path = [[GAM, X, W, L, GAM, K, W, U, X]]
labels = ["$\\Gamma$", "X", "W", "L", "$\\Gamma$", "K", "W", "U", "X"]

#some geom opti cell.
atoms=ase.io.read('/home/physics/phspnt/project_work/local_castep/dia_Si_nondiagsupercells/geom_opti/dia_Si_geomopti.cell')
#phonon grid.
grid=[[4, 0, 0], [0, 4, 0], [0, 0, 4]]

#str location of your potential (asuming GAP, might need to change AtomsToPhonons class to set the Potential)
gap_dir = '/home/physics/phspnt/project_work/quip_stuff/phonon_ndsc/Si_dia/run011_1400sp/'
gap_dia_Si=gap_dir + 'Si_run011_1400sp.xml'
#AtomsToPhonons Class,in this object you will have your phonon eigs.
atp_ndsc = AtomsToPhonons(primitive_cell=atoms, phonon_grid=grid, displacement=0.05, kpath=path, calculator=[gap_dia_Si], plusminus=False)
#this is seperate class to the DOS.
atp_gapPDOS = AtomsToPDOS(primitive_cell=atoms, phonon_grid=grid,
                          displacement=0.05, calculator=[gap_dia_Si])
atp_gapPDOS.get_PDOS([40,40,40])



f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [4, 1]})
a0.plot(atp_ndsc.frequencies[:,:], 'b', label='GAP: ndsc')


#another GAP potential
gap_dir = '/home/physics/phspnt/project_work/quip_stuff/phonon_ndsc/Si_dia/run012/'
gap_dia_Si=gap_dir + 'Si_run012.xml'
atp_ndsc2 = AtomsToPhonons(primitive_cell=atoms, phonon_grid=grid, displacement=0.05, kpath=path, calculator=[gap_dia_Si], plusminus=False)
a0.plot(atp_ndsc2.frequencies[:,:], 'r', label='GAP: shear')

#this is a castep phonon job file. Assumes you use THz in order to parse the castep file. (set that in the param file).
#this castep job is a walk along the same BZ walk.
file = "/home/physics/phspnt/project_work/phonnopy_tut/phonon_castep_direct/Si_dia.castep"
#extract all the eigs 
pfc = PhononFromCastep(castep_file=file, kpath=path, frequency_mask=atp_ndsc.frequencies)
#This is another job, where you do phonons but on a grid isntead.
fileqponts= "/home/physics/phspnt/project_work/local_md_runs/ndsc/Si/Si_dia.castep"
pfcPDOS = PhononFromCastepPDOS(castep_file=fileqponts, atoms_in=atoms)


#ploty things.
a0.plot(pfc.xscale*100*(9-1), pfc.frequencies[:,:], 'k--', label= 'DFT')

weights = np.tile(pfcPDOS.weights , (np.shape(pfcPDOS.frequencies)[1],1)) .flatten(order='F')
a1.hist(pfcPDOS.frequencies.flatten(), weights=weights, bins=50,density=True, alpha=1,
        color='0.5', label='DFT frozen', orientation='horizontal')


AREA= np.trapz(atp_gapPDOS.pdos['projected_dos'][0,:], 
               dx=abs(atp_gapPDOS.pdos['frequency_points'][1]-atp_gapPDOS.pdos['frequency_points'][0]))
a1.plot(atp_gapPDOS.pdos['projected_dos'][0,:]/AREA, atp_gapPDOS.pdos['frequency_points'], 
         color='b', label='GAP frozen')

RMSE = np.sqrt(sum(get_error(atp_ndsc, pfc).flatten()**2)/len(get_error(atp_ndsc, pfc).flatten()))
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
a0.text(0.012, 0.99, f'RMSE = {RMSE:.3f} THz', transform=a0.transAxes, fontsize=20,
        verticalalignment='top', bbox=props)

plt.rcParams["figure.figsize"] =12,10
plt.rcParams.update({'font.size': 22})
a0.set_xlim(atp_gap.normal_ticks[0], atp_gap.normal_ticks[-1])
plt.sca(a0)
plt.xticks(atp_gap.normal_ticks, labels)
a0.grid(axis='x')
a0.axhline(y=0, ls='-', color='k')
a0.set_ylabel('Frequency (THz)')
a0.set_title('Diamond Si', fontsize=28)
a0.set_ylim([-0.25,16])
a1.set_ylim([-0.25,16])
handles, labelsw = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labelsw, handles))
a0.legend(by_label.values(), by_label.keys(), loc=[0.65,0.05], fontsize=20)
f.tight_layout()
plt.savefig('/home/physics/phspnt/transfer/ndsc_images/dia_Si_phonons.eps', format='eps')